In [55]:
from    MCForecastTools        import MCSimulation
from    dotenv                 import load_dotenv
from    dateutil.relativedelta import *
import  calendar
import  datetime
import  pandas as pd
import  hvplot.pandas
import  requests
import  os

%matplotlib inline

In [2]:
# Define constants
CRYPTO_ENDPOINT = "https://api.alternative.me/v2/"

In [3]:
# Let's try to get listings

response = requests.get(
    url=CRYPTO_ENDPOINT + "listings/"
)
crypto_list = response.json()["data"]

In [4]:
crypto_list[0]

{'id': '1', 'name': 'Bitcoin', 'symbol': 'BTC', 'website_slug': 'bitcoin'}

In [5]:
coin_params = {
    "limit" : 0,
    "structure" : "array",
    "sort" : "percent_change_24h",
    "convert" : "USD"
}
response = requests.get(
    url=CRYPTO_ENDPOINT + "ticker/",
    params=coin_params
)
ticker = response.json()


In [43]:
len(ticker["data"])

500

In [44]:
df = pd.DataFrame(ticker["data"])

In [45]:
df.head()

,id,name,symbol,website_slug,rank,circulating_supply,total_supply,max_supply,quotes,last_updated
0,2455,PressOne,PRS,pressone,4053,0,0,2000000000,"{'USD': {'price': 0.03500153, 'volume_24h': 24...",1627299941
1,11953,Unifi Protocol DAO,UNFI,unifi-protocol-dao,418,5716364,5716364,10000000,"{'USD': {'price': 22.99, 'volume_24h': 1191214...",1654646295
2,10400,r/FortNiteBR Bricks,BRICK,brick,5747,0,0,0,"{'USD': {'price': 0.0426695, 'volume_24h': 45,...",1653587919
3,2850,Traxia,TMT,traxia,1724,476441306,476441306,1000000000,"{'USD': {'price': 0.00087897, 'volume_24h': 44...",1653587940
4,11910,TronSecureHybrid,TSCHYBRID,tronsecurehybrid,5122,0,0,499700000,"{'USD': {'price': 6.002e-05, 'volume_24h': 81,...",1627330316


In [62]:
df["date_updated"] = pd.to_datetime(df["last_updated"], unit="s")

In [63]:
df.head()

,id,name,symbol,website_slug,rank,circulating_supply,total_supply,max_supply,quotes,last_updated,date_updated
0,2455,PressOne,PRS,pressone,4053,0,0,2000000000,"{'USD': {'price': 0.03500153, 'volume_24h': 24...",1627299941,2021-07-26 11:45:41
1,11953,Unifi Protocol DAO,UNFI,unifi-protocol-dao,418,5716364,5716364,10000000,"{'USD': {'price': 22.99, 'volume_24h': 1191214...",1654646295,2022-06-07 23:58:15
2,10400,r/FortNiteBR Bricks,BRICK,brick,5747,0,0,0,"{'USD': {'price': 0.0426695, 'volume_24h': 45,...",1653587919,2022-05-26 17:58:39
3,2850,Traxia,TMT,traxia,1724,476441306,476441306,1000000000,"{'USD': {'price': 0.00087897, 'volume_24h': 44...",1653587940,2022-05-26 17:59:00
4,11910,TronSecureHybrid,TSCHYBRID,tronsecurehybrid,5122,0,0,499700000,"{'USD': {'price': 6.002e-05, 'volume_24h': 81,...",1627330316,2021-07-26 20:11:56


In [64]:
df = df.loc[df["total_supply"] != 0]

In [65]:
df.head()

,id,name,symbol,website_slug,rank,circulating_supply,total_supply,max_supply,quotes,last_updated,date_updated
1,11953,Unifi Protocol DAO,UNFI,unifi-protocol-dao,418,5716364,5716364,10000000,"{'USD': {'price': 22.99, 'volume_24h': 1191214...",1654646295,2022-06-07 23:58:15
3,2850,Traxia,TMT,traxia,1724,476441306,476441306,1000000000,"{'USD': {'price': 0.00087897, 'volume_24h': 44...",1653587940,2022-05-26 17:59:00
8,2775,Faceter,FACE,face,963,532000000,532000000,1000000000,"{'USD': {'price': 0.00101752, 'volume_24h': 71...",1653588126,2022-05-26 18:02:06
9,10269,BarterTrade,BART,bartertrade,1207,114804357,114804357,240000000,"{'USD': {'price': 0.00496436, 'volume_24h': 29...",1653588215,2022-05-26 18:03:35
10,10742,Ecoin,ECOIN,ecoin-2,1691,461121563862,461121563862,1000000000000,"{'USD': {'price': 0.00017999, 'volume_24h': 3,...",1666782579,2022-10-26 11:09:39


In [66]:
df.shape

(336, 11)